<a href="https://colab.research.google.com/github/wincya/wincy/blob/master/TransferLearningVGG16ForCalTech101.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the required libraries
import keras
from keras.preprocessing import image
from keras import layers, models, optimizers
from keras.applications import VGG16, ResNet50, DenseNet121

In [2]:
#Define the basemodel as pre-trained VGG16
base_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=True)
base_model.summary()

553467904/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [3]:
#Remove the last FC layer (FC3) and add a new FC layer
X = base_model.layers[-2].output
X = layers.Dense(102, activation='softmax')(X)
#define model input and outputs
base_model = models.Model(inputs=base_model.inputs, outputs=X)

In [4]:
base_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
#Freeze all the layers except last layer
for i in range(len(base_model.layers)-1):
	base_model.layers[i].trainable = False
base_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
#mount google drive into google colab to access the contents
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#specify training and validation data paths
import os
DATA_FOLDER = "/content/drive/My Drive/CalTech101"
# DATA_FOLDER = "CalTech101"
TRAIN_PATH = os.path.join(DATA_FOLDER, "training") # Path for training data
VALID_PATH = os.path.join(DATA_FOLDER, "validation") # Path for validation data
NUMBER_OF_CLASSES = len(os.listdir(TRAIN_PATH)) # Number of classes of the dataset

In [8]:
#define datagenerator to preprocess input data and create train and vlaid generators
batch_size = 8
datagen = image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input) # creating an instance of the data generator
train_generator = datagen.flow_from_directory(TRAIN_PATH, target_size=(224, 224), batch_size=batch_size) # creating the generator for training data
valid_generator = datagen.flow_from_directory(VALID_PATH, target_size=(224, 224), batch_size=batch_size) # creating the generator for validation data


Found 227 images belonging to 102 classes.
Found 1829 images belonging to 102 classes.


In [9]:
#compile and train the model 
base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = base_model.fit_generator(train_generator,validation_data=valid_generator, epochs=1, steps_per_epoch=len(train_generator) / batch_size, validation_steps=len(valid_generator))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


3/3 [==============================] - 704s 263s/step - loss: 4.3104 - accuracy: 0.1542 - val_loss: 9.4782 - val_accuracy: 0.0252


In [10]:
#print maximum validation accuracy
print(max(history.history['val_accuracy']))
print(history.history)

0.025150354951620102
{'loss': [3.652693748474121], 'accuracy': [0.28125], 'val_loss': [9.47815227508545], 'val_accuracy': [0.025150354951620102]}
